# MB-pol Air-Water Simulation Setup

This notebook walks you **step-by-step** through preparing and running a air-water interface simulation using the polarizable **MB-pol** potential with LAMMPS + MBX.  
You'll:  

1. Build the required MBX, MB-pol and LAMMPS input files.  
2. Submit the production NPT run on an HPC cluster.    

Feel free to skim the headings first to see the overall workflow before executing the cells.


### 1 Copy `mbx.json` and  `mbpol.param`
Copies the `mbx.json` and `mbpol.param` into the current folder so LAMMPS can locate MB-pol tables at runtime.  

In [1]:
!cp ../02_bulk_water_simulations/mbx.json .
!cp ../02_bulk_water_simulations/mbpol.param .

## 2  Create the **`start.lmp`** production input

This comprehensive LAMMPS script performs a **constant-volume (NVT)** run with MB-pol:

* Defines user-friendly **variables** for temperature, pressure, time-step, frequencies, etc.  
* Reads the `data.system` file and **includes** `mbpol.param` for the force-field setup.  
* Uses **`fix NPT`** to control temperature and isotropic pressure.  
* Prints thermodynamic output every `thermo_freq` steps and dumps coordinates every `dump_freq`.  
* Writes periodic **restart files** so the job can resume if interrupted.  
* At the end, saves a clean `final_data.system` snapshot.

All parameters are gathered near the top so you can quickly test different conditions.

**Challenge 2**: Go back to Tutorial 1 and modify the input files so it creates a slab of 512 waters having the box dimention `24.8x24.8x100` where the water will be in the middle and in the top and bottom will be vaccume. 


### Description

Generates the multi-line string **`start_lmp_script`** and writes it to `start.lmp`.  
Scroll through the string to see exactly which variables and fixes are used – it doubles as in-file documentation.


In [2]:
start_lmp_script = r"""
# MB-pol water-slab simulation

# Define the required variables
variable        temp        equal  298.0  # simulation temperature
variable        dt          equal  0.5    # time step in fs
variable        thermo_freq equal  8      # themo printing frequeny
variable        dump_freq   equal  8      # traj printing frequeny
variable        nsteps      equal  200000    # number of simulation steps
variable        print_freq  equal  10      # priting freqency
variable        seed        equal  124678 # random seed

timer           timeout 23:50:00 every 1000

processors * * * map xyz

units real
atom_style full
boundary        p p p

# reading the initial configaration
read_data data.system

# Defining atom types for the system
variable        O equal 1
variable        H equal 2

# MB-pol parameter file
include mbpol.param

# Try to Google this command and find out what it does
neighbor        2.0 bin
neigh_modify    every 1 delay 10 check yes

# Defining the timestep
timestep        ${dt}

# 
thermo_style    custom step time temp etotal pe press vol density lx
thermo          ${thermo_freq}
thermo_modify   flush yes

velocity        all create ${temp} ${seed} rot yes dist gaussian

# Printing desierd properites to a file
fix             mbx_print all print ${print_freq} "$(time) $(c_mbx[1]) $(c_mbx[2]) $(c_mbx[3]) $(c_mbx[4]) $(c_mbx[6]) $(c_mbx[6]) $(c_mbx[7]) $(c_mbx[8])" file mbx_thermo.out screen no title "# time e1bpip e2bpip e3bpip e4bpip edisp ebuck eele etot"
fix             thermo_print all print ${print_freq} "$(step) $(time) $(temp) $(etotal) $(ke) $(pe) $(enthalpy) $(density) $(lx) $(ly) $(lz) $(vol) $(pxx) $(pyy) $(pzz) $(press)" file thermo.out screen no title "# step time temp etotal ke pe enthalpy density lx ly lz vol pxx pyy pzz press"

# NVT simulation
fix             NVT all nvt temp ${temp} ${temp} $(100.0*dt)

# For printing the trajectory file
dump            DUMP all custom ${dump_freq} traj_nvt.lammpstrj id type x y z
dump_modify     DUMP sort id

# writing restart files in between the simulation
restart         1000 restart.1 restart.2

# run the simulaiton
run             ${nsteps}

# write teh final configaration to a file
write_data final_data.system nocoeff
"""

with open('start.lmp', 'w') as f:
    f.write(start_lmp_script)

## 4  Create the **`sub.sh`** SLURM submission script 



In [3]:
sub_sh_script = r"""#!/bin/bash

#SBATCH --job-name="air-water"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=16
#SBATCH --mem=32GB
#SBATCH --reservation=QDMS-CPU
#SBATCH -A csd973
#SBATCH --export=ALL
#SBATCH -t 24:00:00

module purge
module load shared slurm/expanse/23.02.7 sdsc/1.0 DefaultModules slurm/expanse/23.02.7 cpu/0.17.3b intel/19.1.3.304/6pv46so intel-tbb/2020.3/lfesfxm intel-mpi/2019.10.317/ezrfjne fftw/3.3.10/tqkvj37

lammps=/expanse/projects/qstore/csd973/bin/lmp_mpi_mbx

export OMP_NUM_THREADS=16

$lammps -in start.lmp
"""

with open('sub.sh', 'w') as f:
    f.write(sub_sh_script)

## 5  Submit the simulation

With all input files in place we can queue the job on the cluster.  


Simply calls **`sbatch sub.sh`** to submit the job to SLURM.  
The command prints the **job ID** so you can monitor progress.


In [4]:
!sbatch sub.sh

Submitted batch job 41156694


Check the job queue periodically to see whether your simulation is still running, waiting or finished.


In [6]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          41156643    shared ion-wate ypolyach  R       8:29      1 exp-2-45
          41156694    shared air-wate ypolyach  R       6:48      1 exp-2-45
